In [ ]:
from grid.eval import *

In [ ]:
worlds, agents = worlds_and_agents(3)

In [ ]:
n_agents = len(agents)
wins  = pd.DataFrame(np.zeros((n_agents, n_agents)), list(agents), list(agents))
games = pd.DataFrame(np.zeros((n_agents, n_agents)), list(agents), list(agents))
soln = None

In [ ]:
while True:
    soln = activelo.solve(games, wins, soln=soln)
    sugg = activelo.suggest(soln)

    results = arena.common.evaluate(worlds, {n: agents[n] for n in sugg})
    games, wins = update(games, wins, results)

    report(soln)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(games)